### Building a Specimen Resource

We will build a Specimen resource using the [Patient](http://fhirtest.b12x.org/r3/Patient/4408) we created and uploaded in the [makePatient](makePatient.ipynb) notebook. We'll include that the specimen was from a buccal swab.

import packages we need...

In [2]:
import json
from fhirclient.r3 import client
import fhirclient.r3.models.specimen as s
import fhirclient.r3.models.patient as p
import fhirclient.r3.models.fhirreference as fr
import fhirclient.r3.models.codeableconcept as cc
import fhirclient.r3.models.coding as co

Create a Specimen object and take a look at its' `.elementProperties()`. See the [makePatient.ipynb](makePatient.ipynb) notebook for explanation of this method.

In [3]:
mySpecimen = s.Specimen()
for eproperty in mySpecimen.elementProperties():
    print(eproperty)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.r3.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.r3.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.r3.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.r3.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.r3.models.narrative.Narrative'>, False, None, False)
('accessionIdentifier', 'accessionIdentifier', <class 'fhirclient.r3.models.identifier.Identifier'>, False, None, False)
('collection', 'collection', <class 'fhirclient.r3.models.specimen.SpecimenCollection'>, False, None, False)
('container', 'container', <class 'fhirclient.r3.models.specimen.SpecimenContainer'>, True, None, False)
('identifie

Which, if any, are required? We'll look at the `not_optional` boolean which is in the last position of the tuple.

Since we'll do this again for other objects, we'll create a little function to determine this.

In [5]:
def whatsRequired(fhirObject):
    required = False
    for eproperty in fhirObject.elementProperties():
        if eproperty[5] == True:
            print(eproperty)
            required = True
    if not required:
        print("Nothing is required")

In [6]:
whatsRequired(mySpecimen)

('subject', 'subject', <class 'fhirclient.r3.models.fhirreference.FHIRReference'>, False, None, True)


Only `subject` is required. Our Subject will be the Patient we created in the previous notebook. We'll use the id of the Patient that was posted to the same server.
We see from `.elementProperties()` that `mySpecimen.subject` is a type of `fhirclient.models.fhirreference.FHIRReference`, so we'll build this.

First, create the `FHIRReference` object.</p>

In [7]:
mySubject = fr.FHIRReference()
for eproperty in mySubject.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.r3.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('display', 'display', <class 'str'>, False, None, False)
('identifier', 'identifier', <class 'fhirclient.r3.models.identifier.Identifier'>, False, None, False)
('reference', 'reference', <class 'str'>, False, None, False)


Which of these is required?

In [8]:
whatsRequired(mySubject)

Nothing is required


None of the elements inside of `subject` are required. We'll use the `reference` and point to the Patient we created, and then take a look to see how `mySubject` looks so far.

In [9]:
mySubject.reference = 'Patient/3004'
mySpecimen.subject = mySubject
import json
print(json.dumps(mySpecimen.as_json(), indent=4))

{
    "subject": {
        "reference": "Patient/3004"
    },
    "resourceType": "Specimen"
}


Now we'll add some collection information to this Specimen. 

In [10]:
myCollection = s.SpecimenCollection()
for eproperty in myCollection.elementProperties():    
    print(eproperty)

('extension', 'extension', <class 'fhirclient.r3.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.r3.models.extension.Extension'>, True, None, False)
('bodySite', 'bodySite', <class 'fhirclient.r3.models.codeableconcept.CodeableConcept'>, False, None, False)
('collectedDateTime', 'collectedDateTime', <class 'fhirclient.r3.models.fhirdate.FHIRDate'>, False, 'collected', False)
('collectedPeriod', 'collectedPeriod', <class 'fhirclient.r3.models.period.Period'>, False, 'collected', False)
('collector', 'collector', <class 'fhirclient.r3.models.fhirreference.FHIRReference'>, False, None, False)
('method', 'method', <class 'fhirclient.r3.models.codeableconcept.CodeableConcept'>, False, None, False)
('quantity', 'quantity', <class 'fhirclient.r3.models.quantity.Quantity'>, False, None, False)


In [11]:
whatsRequired(myCollection)

Nothing is required


None of these are required, but we'll add `bodySite` and `method`. Both are types of `fhirclient.models.codeableconcept.CodeableConcept`. We'll start with `bodySite`, which will be a SNOMED code for buccal space.

In [12]:
myBodySite = cc.CodeableConcept()
for eproperty in myBodySite.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.r3.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('coding', 'coding', <class 'fhirclient.r3.models.coding.Coding'>, True, None, False)
('text', 'text', <class 'str'>, False, None, False)


In [13]:
whatsRequired(myBodySite)

Nothing is required


We'll use `coding` which is a type of `fhirclient.models.coding.Coding`. This is a `is_list` so it must be put into a list. This makes sense because codeable concepts can have more than one coding. Here we'll create just one coding.

In [14]:
myBodySiteCoding = co.Coding()
for eproperty in myBodySiteCoding.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.r3.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('code', 'code', <class 'str'>, False, None, False)
('display', 'display', <class 'str'>, False, None, False)
('system', 'system', <class 'str'>, False, None, False)
('userSelected', 'userSelected', <class 'bool'>, False, None, False)
('version', 'version', <class 'str'>, False, None, False)


In [15]:
whatsRequired(myBodySiteCoding)

Nothing is required


Everything is optional. We'll add `system`, `code`, and `display` to `myBodySiteCoding`. Since `myBodySite` is a codeable concept, it can have multiple codings, so we'll put in a list.

In [16]:
myBodySiteCoding.system = 'http://snomed.info/sct'
myBodySiteCoding.code = '261063000'
myBodySiteCoding.display = 'Buccal space'
myBodySite.coding = [myBodySiteCoding] # is_list for coding is True, so it must be put into a list
print(json.dumps(myBodySite.as_json(), indent=4))

{
    "coding": [
        {
            "code": "261063000",
            "display": "Buccal space",
            "system": "http://snomed.info/sct"
        }
    ]
}


Let's repeat this for `method`, which is another `CodeableConcept`, to indicate it is a swab.

In [17]:
myMethod = cc.CodeableConcept()
myMethodCoding = co.Coding()
myMethodCoding.system = 'http://hl7.org/fhir/v2/0488'
myMethodCoding.code = 'SWA'
myMethodCoding.display = 'Swab'
myMethod.coding = [myMethodCoding] # is_list for coding is True, so must be put into a list
print(json.dumps(myMethod.as_json(), indent=4))

{
    "coding": [
        {
            "code": "SWA",
            "display": "Swab",
            "system": "http://hl7.org/fhir/v2/0488"
        }
    ]
}


Now put `myBodySite` and `myMethod` into `myCollection`, and put `myCollection` into `mySpecimen`.

In [18]:
myCollection.bodySite = myBodySite
myCollection.method = myMethod
mySpecimen.collection = myCollection
print(json.dumps(mySpecimen.as_json(), indent=4))

{
    "collection": {
        "bodySite": {
            "coding": [
                {
                    "code": "261063000",
                    "display": "Buccal space",
                    "system": "http://snomed.info/sct"
                }
            ]
        },
        "method": {
            "coding": [
                {
                    "code": "SWA",
                    "display": "Swab",
                    "system": "http://hl7.org/fhir/v2/0488"
                }
            ]
        }
    },
    "subject": {
        "reference": "Patient/3004"
    },
    "resourceType": "Specimen"
}


Now let's try posting it the FHIR server, using the same settings we used in makePatient.ipynb

In [19]:
settings = {
        'app_id': 'my_web_app',
        'api_base': 'http://fhirtest.b12x.org/r3'
    }
smart = client.FHIRClient(settings=settings)
response = mySpecimen.create(smart.server)
print(json.dumps(response.json(), indent=4))

{
    "resourceType": "Specimen",
    "id": "3010",
    "meta": {
        "versionId": "1",
        "lastUpdated": "2021-06-26T02:51:46.070+00:00"
    },
    "subject": {
        "reference": "Patient/3004"
    },
    "collection": {
        "method": {
            "coding": [
                {
                    "system": "http://hl7.org/fhir/v2/0488",
                    "code": "SWA",
                    "display": "Swab"
                }
            ]
        },
        "bodySite": {
            "coding": [
                {
                    "system": "http://snomed.info/sct",
                    "code": "261063000",
                    "display": "Buccal space"
                }
            ]
        }
    }
}


In [20]:
print(response.headers['Location'])

http://fhirtest.b12x.org/r3/Specimen/3010/_history/1
